In [ ]:
from load_neu import load_neu_data_notnormalized
from sklearn.svm import SVC
from skimage.feature import hog
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report , confusion_matrix

In [ ]:
(x_train,x_test),(y_train,y_test) = load_neu_data_notnormalized(shuffle=True,random_state=21)

In [ ]:
print(x_train.shape,"",y_train.shape)
print(x_test.shape,"",y_test.shape)

In [ ]:
rows = 3
cols = 3
fig, ax = plt.subplots(ncols=cols,nrows=rows,figsize=(rows*3,cols*3))
for i in range(rows):
    for j in range(cols):
        value = np.random.randint(0,len(x_train))
        ax[i][j].imshow(x_train[value].reshape(200,200),cmap=plt.cm.binary)
        ax[i][j].set_title(f'Class {y_train[value]}')
        ax[i][j].axis('off')
fig.tight_layout()

In [ ]:
x_hog_train = []
for x in x_train:
    fd , hogs = hog(x, orientations=16, pixels_per_cell=(16, 16),cells_per_block=(2, 2),
                    visualize=True, multichannel=True,block_norm='L2')
    x_hog_train.append(hogs)
x_hog_train = np.array(x_hog_train)

In [ ]:
x_hog_test = []
for x in x_test:
    fd , hogs = hog(x, orientations=16, pixels_per_cell=(16, 16),cells_per_block=(2, 2),
                    visualize=True, multichannel=True,block_norm='L2')
    x_hog_test.append(hogs)
x_hog_test = np.array(x_hog_test)

In [ ]:
x_hog_train = x_hog_train.reshape(-1,200*200)
x_hog_test = x_hog_test.reshape(-1,200*200)
print(x_hog_train.shape)
print(x_hog_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
scaler = StandardScaler()
x_hog_train = scaler.fit_transform(x_hog_train)
x_hog_test = scaler.fit_transform(x_hog_test)

In [ ]:
rows = 3
cols = 3
fig, ax = plt.subplots(ncols=cols,nrows=rows,figsize=(rows*3,cols*3))
for i in range(rows):
    for j in range(cols):
        value = np.random.randint(0,len(x_train))
        ax[i][j].imshow(x_hog_train[value].reshape(200,200),cmap=plt.cm.binary)
        ax[i][j].set_title(f'Class {y_train[value]}')
        ax[i][j].axis('off')
fig.tight_layout()

In [ ]:
model = SVC(C=1e2)

In [ ]:
model.fit(x_hog_train,y_train)

In [ ]:
model.score(x_hog_test,y_test)

In [ ]:
y_pred = model.predict(x_hog_test)

In [ ]:
label = np.array(['Crazing', 'Inclusion', 'Patches', 'Pitted Surface', 'Rolled In Scale', 'Scratches'])
labels = np.array([' ','Cr', 'In', 'Pa', 'PS', 'RS', 'Sc'])
print(classification_report(y_test,y_pred,target_names=label))

In [ ]:
confusionMatrix = confusion_matrix(y_test,y_pred)
print(confusionMatrix)

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))
im = ax.imshow(confusionMatrix, cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set_xlabel("Actual Label",fontsize=16)
ax.set_ylabel("Predicted Label",fontsize=16)
ax.set_xticklabels(labels,fontsize=18)
ax.set_yticklabels(labels,fontsize=18)
thresh = confusionMatrix.max() / 4.
for i in range(confusionMatrix.shape[0]):
        for j in range(confusionMatrix.shape[1]):
            ax.text(j, i, format(confusionMatrix[i, j], ''),fontsize="14",
                    ha="center", va="center",
                    color="white" if confusionMatrix[i, j] > thresh else "black")
fig.tight_layout(h_pad=0,w_pad=0)
plt.show()